In [9]:
def fetch_historical_data(district, variety):

    historical_data = pd.read_csv('/Users/apple/Downloads/agriculture_data.csv')
    historical_data['Price Date'] = pd.to_datetime(historical_data['Price Date'])
    # Set 'Price Date' as the index of the DataFrame
    historical_data.set_index('Price Date', inplace=True)
    return historical_data

In [ ]:
def fetch_historical_data(district, variety):
    # Load the data
    historical_data = pd.read_csv('/Users/apple/Downloads/agriculture_data.csv')
    
    # Convert 'Price Date' to datetime
    historical_data['Price Date'] = pd.to_datetime(historical_data['Price Date'])
    
    # Filter for the specific district and variety
    historical_data = historical_data[(historical_data['District Name'] == district) & 
                                      (historical_data['Variety'] == variety)]
    
    # Aggregate the mean price for each date
    aggregated_data = historical_data.groupby('Price Date').agg({'Capped Price': 'mean'}).reset_index()
    
    # Set 'Price Date' as the index of the DataFrame
    aggregated_data.set_index('Price Date', inplace=True)
    
    return aggregated_data


In [13]:
import joblib

# Load the trained models
model_dict = joblib.load('/Users/apple/Downloads/all_models.pkl')


/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using ver

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

# Load the trained models
model_dict = joblib.load('/Users/apple/Downloads/all_models.pkl')

import pandas as pd

def fetch_historical_data(district, variety):
    # Load the historical data
    historical_data = pd.read_csv('/Users/apple/Downloads/agriculture_data.csv')

    # Convert 'Price Date' to datetime
    historical_data['Price Date'] = pd.to_datetime(historical_data['Price Date'])
    
    # Aggregate the mean price for each date, district, and variety
    historical_data = (historical_data
                       .groupby(['Price Date', 'District Name', 'Variety'])
                       .agg({'Capped Price': 'mean'})  # Change 'Capped Price' to the relevant column name
                       .reset_index())

    # Set 'Price Date' as the index of the DataFrame
    historical_data.set_index('Price Date', inplace=True)

    # Filter the historical data for the specified district and variety
    historical_data = historical_data[(historical_data['District Name'] == district) & 
                                      (historical_data['Variety'] == variety)]
    
    return historical_data



/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using ver

In [41]:
def prepare_input_for_prediction(historical_data):
    # Check if the historical data DataFrame is empty
    if historical_data.empty:
        raise ValueError("Historical data is empty. Please check the data source.")

    # Resample and prepare historical data
    historical_data = historical_data['Capped Price'].resample('M').mean().ffill()

    # Create a DataFrame for processing
    df = pd.DataFrame({'Value': historical_data})

    # Create lag features
    for lag in range(1, 13):  # Lag features for the past 12 months
        df[f'lag_{lag}'] = df['Value'].shift(lag)

    # Add rolling features
    df['rolling_mean'] = df['Value'].rolling(window=3).mean()  # 3-month rolling mean
    df['rolling_std'] = df['Value'].rolling(window=3).std()    # 3-month rolling standard deviation

    # Drop rows with NaN values created by lags and rolling calculations
    df.dropna(inplace=True)

    if df.empty:
        raise ValueError("Not enough data to create lag features and rolling statistics. Please check the historical data.")

    # Apply Min-Max scaling
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['Value']])

    # Replace the original values with scaled values
    df['Value'] = scaled_data.flatten()

    return df, scaler


In [42]:
def predict_price(district, variety, future_date):
    # Get historical data for the district and variety
    historical_data = fetch_historical_data(district, variety)

    # Prepare the input data based on the historical data
    try:
        df, scaler = prepare_input_for_prediction(historical_data)
    except ValueError as e:
        print(e)
        return None

    # Retrieve the model for the specified district and variety
    model_info = model_dict.get((district, variety))

    if model_info is None:
        print("No model found for the specified district and variety.")
        return None

    model = model_info['model']

    # Use the most recent row of df (latest data) as input for the model
    if df.empty:
        print("No valid input data for prediction.")
        return None

    X_new = df.iloc[-1:].drop(columns=['Value'])

    # Make the prediction using the loaded model
    predicted_value = model.predict(X_new)

    # Inverse transform to convert the scaled value back to the original price scale
    predicted_price = scaler.inverse_transform(predicted_value.reshape(-1, 1))

    return predicted_price.flatten()[0]  # Flatten to get the scalar value


In [43]:
# Example usage
district = 'Mysore'
variety = 'Tomato'
future_date = '2024-01-01'

predicted_price = predict_price(district, variety, future_date)
if predicted_price is not None:
    print(f"The predicted price for {district} - {variety} on {future_date} is: {predicted_price:.2f} Rs./Quintal")


The predicted price for Mysore - Tomato on 2024-01-01 is: 1483.14 Rs./Quintal


/var/folders/3_/121mk5v92fsgc66gxg541l4w0000gn/T/ipykernel_6279/1431705064.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  historical_data = historical_data['Capped Price'].resample('M').mean().ffill()
